## BETTER WAY 26 - 믹스인 유틸리티 클래스에만 다중 상속을 사용하자
## Item26 - Use Multiple Inheritance Only for Mix-in Uitility Classes

파이썬은 다중 상속을 다루기 쉽게하는 기능을 내장한 객체 지향 언어이지만, 다중 상속은 아예 하지 않는 것이 좋다. 

다중 상속으로 얻는 편리함과 캡슐화가 필요하다면 믹스인(Mix-in)을 작성하는 방법이 있다. 믹스인이란 클래스에서 제공해야하는 추가적인 메소드만 정의하는 작은 클래스를 말한다. 믹스인 클래스는 자체의 인스턴스 속성(attribute)을 정의하지 않으며 `__init__` 생성자를 호출하도록 요구하지 않는다.

믹스인들을 조합하고 계층으로 구성하면 반복 코드를 최소화하고 재사용성을 극대화할 수 있다.

예를 들어 파이썬 객체를 메모리 내부 표현에서 직렬화(serialization)용 딕셔너리로 변환하는 기능이 필요하다고 해보자. 이 기능을 모든 클래스에서 사용할 수 있게 범용으로 작성하는 건 어떨까?

In [1]:
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

다음은 바이너리 트리(binary tree)를 딕셔너리로 표현하기 위해 믹스인을 사용하는 예제 클래스이다.

In [2]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [3]:
tree = BinaryTree(10, 
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


믹스인의 가장 큰 장점은 범용 기능을 교체할 수 있게 만들어서 필요할 때 동작을 오버라이드할 수 있다는 점이다.

In [4]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent

도저히 이해가 되지 않는다. 추후에 공부를 더 한뒤 마저 포스팅 하겠다...ㅜㅜ 후.. ㅜㅜ